In [1]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset, AgeDatasetKL
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#########################
import pandas as pd
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
aug = CSVUtils.get_df_from_csv("./augumentation_balanced_remove.csv", "./newAugmentationDataset/")
df_train_aug = pd.concat([df_train, aug], ignore_index=True)
df_train_aug = df_train_aug.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################

from torchvision import transforms
import torch
import numpy as np

transform_func = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.RandAugment(2, 9),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

transform_func_val = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# Implementazione di un Dataset utilizzando "CustomDataset" per l'architettura con Film
cd_train_balanced = AgeDatasetKL(df_train_aug, path_col="path", label_col="age", label_function="Linear", 
                                transform_func=transform_func)
cd_train_balanced.set_n_classes(81)
cd_train_balanced.set_starting_class(1)
dm_train_balanced = CustomDataLoader(cd_train_balanced)
dl_train_balanced, sampler = dm_train_balanced.get_balanced_class_dataloader2(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                                            batch_size=64, num_workers=1, prefetch_factor=2)
sampler.n_batches = 10
sampler.p = np.array([1, 0, 0, 0, 0, 0, 0, 0])

cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_func_val)
cd_val.set_n_classes(81)
cd_val.set_starting_class(1)
validator = Validator(cd_val, AgeConversion.ArgMaxAge, 32, num_workers=4, prefetch_factor=2)

In [4]:
for d in dl_train_balanced:
    print(d)
    break

[tensor([[[[ 0.0569,  0.0569,  0.0398,  ..., -0.2342, -0.2856, -0.2513],
          [ 0.0398,  0.0569,  0.0569,  ..., -0.2856, -0.2513, -0.2856],
          [ 0.0569,  0.0398, -0.0116,  ..., -0.3027, -0.2856, -0.2342],
          ...,
          [-2.1179, -2.1179, -2.1179,  ...,  0.7077,  0.6906,  0.6734],
          [-2.1179, -2.1179, -2.1179,  ...,  0.6392,  0.6392,  0.6221],
          [-2.1179, -2.1179, -2.1179,  ...,  0.6049,  0.5878,  0.5707]],

         [[-0.0049,  0.0126,  0.0301,  ..., -0.3025, -0.2850, -0.2675],
          [-0.0049, -0.0049,  0.0301,  ..., -0.3375, -0.3200, -0.2850],
          [ 0.0126,  0.0301,  0.0126,  ..., -0.3725, -0.3375, -0.3025],
          ...,
          [-2.0357, -2.0357, -2.0357,  ...,  0.5028,  0.5028,  0.5028],
          [-2.0357, -2.0357, -2.0357,  ...,  0.4853,  0.4678,  0.4328],
          [-2.0357, -2.0357, -2.0357,  ...,  0.4678,  0.4328,  0.3803]],

         [[ 0.2522,  0.3742,  0.4614,  ...,  0.1128,  0.0082, -0.0092],
          [ 0.3568,  0.4439, 

In [5]:
print(d[1][0])
t: torch.Tensor = d[1][1]
# t.sum(dim=-1)
print(torch.round(AgeConversion.EVAge(d[1][1])))
print(AgeConversion.EVAge(d[1][1] / t.sum(dim=-1, keepdim=True).expand(-1, 81)))

tensor([2, 3, 8, 0, 7, 7, 7, 7, 2, 8, 6, 6, 7, 4, 6, 9, 4, 4, 2, 5, 0, 3, 1, 8,
        2, 5, 1, 3, 7, 7, 9, 8, 1, 8, 9, 8, 6, 2, 6, 2, 4, 3, 6, 9, 9, 6, 4, 2,
        6, 4, 5, 2, 7, 8, 1, 6, 9, 5, 2, 7, 6, 5, 2, 3])
tensor([2., 3., 8., 1., 7., 7., 7., 7., 2., 8., 6., 6., 7., 4., 6., 9., 4., 4.,
        2., 5., 1., 3., 1., 8., 2., 5., 1., 3., 7., 7., 9., 8., 1., 8., 9., 8.,
        6., 2., 6., 2., 4., 3., 6., 9., 9., 6., 4., 2., 6., 4., 5., 2., 7., 8.,
        1., 6., 9., 5., 2., 7., 6., 5., 2., 3.], dtype=torch.float64)
tensor([2.0139, 3.0005, 8.0000, 0.5201, 7.0000, 7.0000, 7.0000, 7.0000, 2.0139,
        8.0000, 6.0000, 6.0000, 7.0000, 4.0000, 6.0000, 9.0000, 4.0000, 4.0000,
        2.0139, 5.0000, 0.5201, 3.0005, 1.1294, 8.0000, 2.0139, 5.0000, 1.1294,
        3.0005, 7.0000, 7.0000, 9.0000, 8.0000, 1.1294, 8.0000, 9.0000, 8.0000,
        6.0000, 2.0139, 6.0000, 2.0139, 4.0000, 3.0005, 6.0000, 9.0000, 9.0000,
        6.0000, 4.0000, 2.0139, 6.0000, 4.0000, 5.0000, 2.0139, 7.0000, 8

In [6]:
t.shape

torch.Size([64, 81])

In [7]:
np.tile(np.sum(t.numpy(), axis=-1, keepdims=True), (1, 81))

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [8]:
import numpy as np
def _to_kl_labels(y, n_classes):
    std = 1.0
    _y = np.arange(n_classes)
    return 1/(std * np.sqrt(2*np.pi)) * np.exp(-np.square(_y-y) / (2*std**2))

In [9]:
t = _to_kl_labels(1, 81)

In [10]:
t

array([2.41970725e-001, 3.98942280e-001, 2.41970725e-001, 5.39909665e-002,
       4.43184841e-003, 1.33830226e-004, 1.48671951e-006, 6.07588285e-009,
       9.13472041e-012, 5.05227108e-015, 1.02797736e-018, 7.69459863e-023,
       2.11881925e-027, 2.14638374e-032, 7.99882776e-038, 1.09660656e-043,
       5.53070955e-050, 1.02616307e-056, 7.00418213e-064, 1.75874954e-071,
       1.62463604e-079, 5.52094836e-088, 6.90202942e-097, 3.17428155e-106,
       5.37056037e-116, 3.34271444e-126, 7.65392974e-137, 6.44725997e-148,
       1.99788926e-159, 2.27757748e-171, 9.55169454e-184, 1.47364613e-196,
       8.36395161e-210, 1.74636626e-223, 1.34141967e-237, 3.79052640e-252,
       3.94039628e-267, 1.50690472e-282, 2.12000655e-298, 1.09722105e-314,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0

In [11]:
AgeConversion.EVAge(torch.unsqueeze(torch.tensor(_to_kl_labels(1, 81)), dim=0) / torch.unsqueeze(torch.tensor(_to_kl_labels(1, 81)), dim=0).sum())

tensor([1.1294], dtype=torch.float64)

In [12]:
AgeConversion.EVAge(torch.unsqueeze(torch.tensor(_to_kl_labels(0, 81)), dim=0))

tensor([0.3638], dtype=torch.float64)

In [8]:
from ResNetFilmed.resnet import ResNetFiLMed, BackBone, ResNetNotFiLMed, DoNothingLayer
from torchvision.models import resnet18, ResNet18_Weights, efficientnet_b0, EfficientNet_B0_Weights
import torch
from torch import optim
import torch.nn.functional as F
from torch import nn

####################################################
EPOCHS = 24
####################################################

backbone = resnet18(ResNet18_Weights.IMAGENET1K_V1)
backbone.fc = DoNothingLayer()
backbone.train()
backbone.requires_grad_(True)
backbone.to("cuda")
model_age = ResNetNotFiLMed(backbone, 81)
model_age.load_state_dict(torch.load("./model_age_classification_simple.pt", map_location="cuda:0"))

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


<All keys matched successfully>

In [9]:
def forward_function(x):
    out = model_age(x)
    out = F.softmax(out, dim=-1)
    return out

print(validator.validate_ext3(forward_function))

  0%|          | 0/4493 [00:00<?, ? batch/s]

In [5]:
import numpy as np
def get_centers(outs, ys, old):
    centers_for_age = {x: [] for x in range(81)}

    for out, y in zip(outs, ys):
        for _out, _y in zip(out, y):
            centers_for_age[int(_y)].append(_out.detach().cpu().numpy())

    for i in centers_for_age:
        if len(centers_for_age[i]) > 0:
            centers_for_age[i] = torch.tensor(np.array(centers_for_age[i])).mean(dim=0).to("cuda")
        else:
            centers_for_age[i] = old[i]
    return centers_for_age

def update_centers(old, new, alpha=0.5):
    for i in new:
        new[i] = new[i] - alpha*(new[i] - old[i])
    return new

def get_centers_loss(out, y, centers_for_age):
    loss = None
    for _out, _y in zip(out, y):
        if loss is None:
            loss = torch.mean(torch.square(_out - centers_for_age[int(_y)]))
        else:    
            loss += torch.mean(torch.square(_out - centers_for_age[int(_y)]))
    return loss.to("cuda")

In [6]:
centers_for_age = {x: torch.zeros(size=(512,), device="cuda") for x in range(81)}
best_val_aar = val_aar

for e in range(EPOCHS):
    with tqdm(zip(dl_train, dl_train_balanced), unit=" batch") as tepoch:
        for batch, batch_balanced in tepoch:
            opt.zero_grad()
            x, y = batch
            x_bal, y_bal = batch_balanced

            x = x.to("cuda")
            y_age: torch.Tensor = y[0].to("cuda")
            y_age_kl: torch.Tensor = y[1].to("cuda")

            x_bal = x_bal.to("cuda")
            y_age_bal: torch.Tensor = y_bal[0].to("cuda")
            y_age_kl_bal: torch.Tensor = y_bal[1].to("cuda")

            out_rep, out_age = model_age.forward_with_repr(x)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)
            out_age = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out_age).to("cuda")
            loss_age = torch.mean(torch.abs(y_age - out))
            loss = loss_age_kl + loss_age

            out_rep_bal, out_age_bal = model_age.forward_with_repr(x_bal)
            loss_age_kl_bal: torch.Tensor = kl(F.log_softmax(out_age_bal, dim=-1), y_age_kl_bal)
            out_age_bal = F.softmax(out_age_bal, dim=-1)
            out_bal = AgeConversion.EVAge(out_age_bal).to("cuda")
            loss_age_bal = torch.mean(torch.abs(y_age_bal - out_bal))
            loss_bal = loss_age_kl_bal + torch.square(loss_age_bal - 2.0)

            loss_repr = get_centers_loss(out_rep, y_age, centers_for_age)
            loss_repr_bal = get_centers_loss(out_rep_bal, y_age_bal, centers_for_age)

            total_loss = loss + loss_bal + loss_repr + loss_repr_bal

            total_loss.backward()
            opt.step()
            scheduler.step()

            centers_for_age = update_centers(centers_for_age, get_centers((out_rep, out_rep_bal), (y_age, y_age_bal), centers_for_age), alpha=0.5)

            tepoch.set_postfix(loss_age_kl=loss_age_kl.detach().cpu().numpy(), loss_age=loss_age.detach().cpu().numpy(),
                                loss_age_kl_bal=loss_age_kl_bal.detach().cpu().numpy(), loss_age_bal=loss_age_bal.detach().cpu().numpy(),
                                loss_repr=loss_repr.detach().cpu().numpy(), loss_repr_bal=loss_repr_bal.detach().cpu().numpy(),
                                total_loss=total_loss.detach().cpu().numpy())

    def forward_function(x):
        out = model_age(x)
        out = F.softmax(out, dim=-1)
        return out
 
    ae, mae_, val_aar, val_aar_old = validator.validate_ext2(forward_function)
    print(ae, mae_, val_aar, val_aar_old)

    if best_val_aar < val_aar:
        best_val_aar = val_aar
        torch.save(model_age.state_dict(), "./model_age_feature_simple_no_loss.pt")
        print("Saved model")

233 batch [01:47,  2.20 batch/s, loss_age=2.5409758, loss_age_bal=2.5297728, loss_age_kl=1.2492404040357374, loss_age_kl_bal=1.21976906498012, loss_repr=0.61008906, loss_repr_bal=0.5650206, total_loss=6.465754131539128]   